# Audio books business case

## Business Problem

We have data from an audio books app.  Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.
The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertising to him/her. If we can focus our efforts solely on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.


## Data Preprocessing. Balance the dataset. Create 3 datasets: training, validation, and test. Save the newly created sets in a tensor friendly format (.npz)

In [1]:
! pip install sklearn

In [2]:
import numpy as np
from sklearn import preprocessing

In [3]:
raw_csv_data = np.loadtxt('C:\\Users\\Petar\\Desktop\\Audiobooks_data.csv',delimiter=',')

In [4]:
# The inputs are all columns in the csv, except for the first one [:,0](the arbitrary customer IDs that bear no useful information)
# and the last one [:,-1] (which is our targets)
unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [5]:
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [6]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [7]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [8]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]
# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count
# Create variables that record the inputs and targets for training
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]
# Create variables that record the inputs and targets for validation.
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]
# Create variables that record the inputs and targets for test.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]
# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1807.0 3579 0.504889633975971
216.0 447 0.48322147651006714
214.0 448 0.47767857142857145


### Save the three datasets in .npz format

In [9]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [10]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

In [12]:
# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [13]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
tf.keras.layers.Dense(output_size, activation='softmax')
])
### Choose the optimizer and the loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
### Training
# set the batch size
batch_size = 100
# set a maximum number of training epochs
max_epochs = 100
# set an early stopping mechanism
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
# fit the model
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
epochs=max_epochs,
callbacks=[early_stopping], # early stopping
validation_data=(validation_inputs, validation_targets), # validation data
verbose = 2 # making sure we get enough information about the training process
          )  

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.5236 - accuracy: 0.8142 - val_loss: 0.4282 - val_accuracy: 0.8613
Epoch 2/100
3579/3579 - 0s - loss: 0.3664 - accuracy: 0.8740 - val_loss: 0.3455 - val_accuracy: 0.8770
Epoch 3/100
3579/3579 - 0s - loss: 0.3212 - accuracy: 0.8854 - val_loss: 0.3117 - val_accuracy: 0.8904
Epoch 4/100
3579/3579 - 0s - loss: 0.2977 - accuracy: 0.8885 - val_loss: 0.2961 - val_accuracy: 0.8926
Epoch 5/100
3579/3579 - 0s - loss: 0.2822 - accuracy: 0.8947 - val_loss: 0.2819 - val_accuracy: 0.8926
Epoch 6/100
3579/3579 - 0s - loss: 0.2741 - accuracy: 0.8963 - val_loss: 0.2721 - val_accuracy: 0.8993
Epoch 7/100
3579/3579 - 0s - loss: 0.2654 - accuracy: 0.8994 - val_loss: 0.2702 - val_accuracy: 0.9016
Epoch 8/100
3579/3579 - 0s - loss: 0.2618 - accuracy: 0.8997 - val_loss: 0.2655 - val_accuracy: 0.9038
Epoch 9/100
3579/3579 - 0s - loss: 0.2577 - accuracy: 0.9000 - val_loss: 0.2644 - val_accuracy: 0.9105
Epoch 10/100
3579/3579 - 0

## Test the model

In [14]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 477us/sample - loss: 0.2487 - accuracy: 0.9040


In [15]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.25. Test accuracy: 90.40%
